In [0]:
df_orders = spark.sql("SELECT * FROM mycatalog.olist_ecommerce_bronze.orders")
df_orders.display(df_orders)

# Data Type Standardization

- Convert timestamps properly

In [0]:
df_orders.printSchema()

In [0]:
from pyspark.sql.functions import *

In [0]:


df = df_orders\
  .withColumn("order_purchase_timestamp", col("order_purchase_timestamp").cast("timestamp"))\
   .withColumn("order_approved_at", col("order_approved_at").cast("timestamp"))\
   .withColumn("order_delivered_carrier_date", col("order_delivered_carrier_date").cast("timestamp"))\
   .withColumn("order_delivered_customer_date", col("order_delivered_customer_date").cast("timestamp"))\
   .withColumn("order_estimated_delivery_date", col("order_estimated_delivery_date")) 

### CONVERT THE ORDER STATUS TO UPPER AND TRIM

- Prevents grouping errors
- Ensures consistent KPI calculations
- Avoids duplicates like "Delivered" vs "delivered"

In [0]:
df = df\
    .withColumn("order_status", upper(trim(col("order_status"))))

### REMOVE DUPLICATE ORDERS

Justification:

- Protect against ingestion duplication
- Orders should be unique
- Prevents double revenue counting

In [0]:
df = df.dropDuplicates(["order_id"])

Null Handling & Data Quality Enforcement

- Remove invalid primary keys
- Validate delivery logic
   - Enforces business correctness
   - Prevents incorrect SLA metrics in Gold

In [0]:
df = df.filter(col("order_id").isNotNull())

In [0]:
df = df.filter(
    ~( 
        (col("order_status") == "DELIVERED") &
        (col("order_delivered_customer_date").isNull())
    )
)


# Create Derived Business Columns
- Delivery Duration (Actual)

   Justification:
  - Frequently used metric
  - Avoid recalculating in Gold
  - Keeps logic centralized

- Estimated vs Actual Delay

  Justification:
   - Important KPI
   - Operational analytics
   - Performance measurement

   Positive → Late , Negative → Early

- On-Time Flag
   Justification:
     - Makes BI simpler
     - Avoids recalculating logic in dashboards









In [0]:
df = df\
    .withColumn("delivery_days", 
        datediff(
              col("order_delivered_customer_date"),
              col("order_purchase_timestamp")
          )      
    )

In [0]:
df = df\
    .withColumn("delivery_delay_days",
        datediff(
            col("order_delivered_customer_date"),
            col("order_estimated_delivery_date")
        ))

In [0]:
df = df\
    .withColumn("is_late", 
        when(col("delivery_delay_days") > 0, 1).otherwise(0)
    )


### Join with silver customer_geoplocation Tables (Light Enrichment Allowed)

In [0]:
df_customers = spark.sql("SELECT * FROM mycatalog.olist_ecommerce_silver.customer_geolocation")
df_orders_enr = df.join(df_customers, "customer_id", "left").select(
    "customer_id",
    "order_id",
    "order_status",
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "delivery_days",
    "delivery_delay_days",
    "is_late",
    "customer_unique_id",
    "customer_zip_code_prefix",
    "customer_city",
    "customer_state",
    "geolocation_lat",
    "geolocation_lng"
)

### SLA & Operational Classification

Justification:

 - Business-friendly categories
 - Improves Gold layer simplicity
 - Improves Power BI usability

In [0]:
df_enr_final = df_orders_enr\
    .withColumn("delivery_performance", 
        when(col("delivery_delay_days") >=0 , "LATE")\
        .when(col("delivery_delay_days") <0 , "ONTIME")
        .otherwise("PENDING")
    )

In [0]:
df_enr_final.write.format("delta").mode("overwrite").saveAsTable("mycatalog.olist_ecommerce_silver.orders_enriched")